In [1]:
pip install Distance

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 23.0.1
[notice] To update, run: C:\Users\yoga\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
from sklearn.cluster import AffinityPropagation
import distance
    
#words = "hello jello bye tye".split(" ") #Replace this line
words=["hello","jello",'bye',"tie"]
words = np.asarray(words) #So that indexing with a list will work
lev_similarity = -1*np.array([[distance.levenshtein(w1,w2) for w1 in words] for w2 in words])

affprop = AffinityPropagation(affinity="precomputed", damping=0.5)
affprop.fit(lev_similarity)
for cluster_id in np.unique(affprop.labels_):
    exemplar = words[affprop.cluster_centers_indices_[cluster_id]]
    cluster = np.unique(words[np.nonzero(affprop.labels_==cluster_id)])
    cluster_str = ", ".join(cluster)
    print(" - *%s:* %s" % (exemplar, cluster_str))

 - *hello:* hello, jello
 - *tie:* bye, tie


In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
#import nltk.metrics.distance as distance
#import Pycluster as PC
import numpy as np
from sklearn.cluster import AffinityPropagation
import distance

Collecting Common Passwords and storing them in a csv file

In [3]:
# Scraping Website 1
url1 = "https://en.wikipedia.org/wiki/Wikipedia:10,000_most_common_passwords"
webdata  = requests.get(url1).text
soup = BeautifulSoup(webdata,"html.parser")
cols = soup.find_all('div',class_='div-col')
data = {"Common Passwords":[]}
for col in cols:
    passwds=col.find_all('li')
    for passwd in passwds:
        if passwd.text.strip()!='':
            data["Common Passwords"].append(passwd.text)
#Scraping Website 2
url2 = 'https://github.com/OWASP/passfault/blob/master/wordlists/wordlists/10k-worst-passwords.txt'
webdata  = requests.get(url2).text
soup = BeautifulSoup(webdata,"html.parser")
passwds = soup.find_all('td',class_="blob-code blob-code-inner js-file-line")
for passwd in passwds:
    data["Common Passwords"].append(passwd.text)
#Scraping Website 3
url3='https://github.com/danielmiessler/SecLists/blob/master/Passwords/Common-Credentials/10-million-password-list-top-100000.txt'
webdata = requests.get(url3).text
soup = BeautifulSoup(webdata,"html.parser")
passwds = soup.find_all('td',class_='blob-code blob-code-inner js-file-line')
for passwd in passwds:
    data["Common Passwords"].append(passwd.text)    

#Writing all unique passwrods into a csv file    
common_password = pd.DataFrame.from_dict(data)
common_password = common_password[~common_password['Common Passwords'].str.isdigit()]
common_password = common_password.drop_duplicates(subset='Common Passwords', keep="first")
common_password = common_password.reset_index(drop=True)
common_password = common_password.sample(frac=1)
common_password.to_csv('common_passwords.csv')
words=common_password['Common Passwords'].tolist()
print(len(words))

In [ ]:
#clustering using AffinityPropagation
import numpy as np
from sklearn.cluster import AffinityPropagation
import distance
words=common_password['Common Passwords'].tolist()
words=words[:4000]
words = np.asarray(words) #So that indexing with a list will work
lev_similarity = -1*np.array([[distance.levenshtein(w1,w2) for w1 in words] for w2 in words])

affprop = AffinityPropagation(affinity="precomputed", damping=0.5)
affprop.fit(lev_similarity)
clusters=[]
for cluster_id in np.unique(affprop.labels_):
    clus=[]
    exemplar = words[affprop.cluster_centers_indices_[cluster_id]]
    clus.append(exemplar)
    cluster = np.unique(words[np.nonzero(affprop.labels_==cluster_id)])
    for p in cluster:
        if p!=exemplar:
            clus.append(p)
    clusters.append(clus)
    cluster_str = ", ".join(cluster)
    print(" - *%s:* %s" % (exemplar, cluster_str))

In [ ]:
clusters #Nested list in which each element contains cluster of passwords
temp=clusters

In [ ]:
#Finding different patterns present in each cluster and the frequency of these patterns
from difflib import SequenceMatcher 
def longest_Substring(s1,s2):
    
    seq_match = SequenceMatcher(None,s1,s2) 
    match = seq_match.find_longest_match(0, len(s1), 0, len(s2)) 
    # return the longest substring 
    if (match.size!=0):
        return (s1[match.a: match.a + match.size])  
    else: 
        return False
patternInClusters=[]
for i in range(len(temp)):
    patternInClusters.append({})
    exemplar=temp[i][0]
    for j in temp[i][1:]:
        x=longest_Substring(exemplar,j)
        if x!=False and len(x)>1:
            if x in patternInClusters[i]:
                patternInClusters[i][x]+=1
            else:
                patternInClusters[i][x]=1

patternInClusters

In [ ]:
#adding the most common patterns in each cluster to a final pattern list
patterns=[]
for sampleDict in patternInClusters:
    if sampleDict=={}:
        continue
    itemMaxValue = max(sampleDict.items(), key=lambda x: x[1])
    #print('Maximum Value in Dictionary : ', itemMaxValue[1])
    listOfKeys = list()
    # Iterate over all the items in dictionary to find keys with max value
    for key, value in sampleDict.items():
        if value == itemMaxValue[1]:
            listOfKeys.append(key)
    for i in listOfKeys:
        if i not in patterns:
            patterns.append(i)
patterns

In [ ]:
#Implementing Algorithm to generate passwords
import random
words = input("Enter five words(give space between each word): ").split()
numbers = input("Enter two numbers(give space between each number): ").split()
num_passwords = int(input("Enter number of passwords required:"))
special_chars = ['@','$','!','#','%','&','(',')','0','3','8','<','|']
punc = ['@','*','+','-',':','”','/','\\','~','?','[',']','{','}','$','!','#','%','&','(',')','_' ,'<','|']
alphabets = ['a','s','i','r','x','q','c','j','o','e','b','k','l']
c2s = {}
for i in range(13):
    c2s[alphabets[i]]=special_chars[i]
def randomly_capitalize(s):
    result = ''
    for char in s:
        flag = random.randint(0, 1)
        if flag == 0:
            result += char.upper()
        else:
            result += char.lower()
    return result
def merge_randomly(str1,str2):
    result = ''
    flag = random.randint(0, 1)
    if flag == 0:
        result = str1 + str2
    else:
        result = str2 + str1
    return result,flag
def charToSpecial(s):
    for char in s:
        if char in c2s:
            s = s.replace(char,c2s[char])
    return s
def insert_number(s,n,f,str1,str2):
    flag = random.randint(0, 2)
    if flag == 0:
        return n+s
    elif flag == 2:
        return s+n
    else:
        if f==0:
            return s[:len(str1)]+n+s[len(str1):]
        else:
            return s[:len(str2)]+n+s[len(str2):]
def add_punctuation(s):
    return s+random.choice(punc)
passwords_generated=[]
while num_passwords>0:
    temp = list(words)
    string1 = random.choice(temp)
    temp.remove(string1)
    string2 = random.choice(temp)
    number = random.choice(numbers)
    string1,string2 = randomly_capitalize(string1),randomly_capitalize(string2)
    final_string,f = merge_randomly(string1,string2)
    final_string = charToSpecial(final_string)
    final_string = insert_number(final_string,number,f,string1,string2)
    final_string = add_punctuation(final_string)
    C = len(final_string)
    if C < 8:
        for i in range(8 - C):
            final_string += random.choice(special_chars)
    passwords_generated.append(final_string)
    print("Final Passowrd:",final_string)
    num_passwords -= 1

In [ ]:
#To analyze given/generated password
import re
patterns=['shadow',
 'a199',
 'dima199',
 '91',
 '1991',
 'pun',
 'EY',
 '1234',
 '123',
 'eirf',
 'nhei',
 'dj',
 'jhj',
 'jy',
 'way',
 'amst',
 'te',
 'erda',
 'sterda',
 'al',
 'ala',
 'jordan',
 'cho',
 'ol',
 'evild',
 'evi',
 'evil',
 'dead',
 'ad',
 'ea',
 'vil',
 'ld',
 '000',
 'blue',
 'aaa',
 'a2',
 'str',
 'moni',
 'er',
 'ojo11',
 'mo',
 'mojo',
 '11',
 'zxc',
 'zxcv',
 'love',
 'br',
 'bryan',
 'an',
 'azz',
 'ova',
 'lan',
 'rate',
 'espera',
 'esper',
 'ca',
 'on',
 'su',
 'ge',
 'rna',
 'rn',
 'fur',
 'ur',
 'ace',
 'butt',
 'bur',
 'ina',
 'jack',
 'auto',
 'ill',
 'tor',
 'ector',
 'ecto',
 'or',
 'ect',
 've',
 'cto',
 'ctor',
 'and',
 'fre',
 '12',
 'hot',
 'ot',
 'cial',
 'pecial',
 'pe',
 'pi',
 '19',
 'oleg1',
 'shir',
 'sh',
 'man',
 'da',
 'dais',
 'ab',
 'ba',
 'angel',
 'arky',
 'ra',
 'ar',
 'ake',
 'ma',
 '12345678',
 'cooter',
 'co',
 'scooter',
 '6chevy',
 'killer',
 'qwerty',
 'ggy',
 'master',
 'ers',
 'head',
 'david',
 'ing',
 'ga',
 'lon',
 'ck',
 'dick',
 'long',
 'ng',
 'count',
 'ou',
 'oun',
 'ted',
 'ire',
 'oo',
 'fire',
 'artin',
 'martin',
 'in1',
 'ker1',
 'acker',
 '23',
 'poppy',
 'pp',
 'ppy123',
 'be',
 'zer',
 'ee',
 'thunder',
 'dan',
 'ana',
 'ho',
 'ota',
 'tac',
 'vol',
 'el',
 'elvis',
 's99',
 'elvi',
 'ri',
 'alex',
 'se',
 'hi',
 'pimp',
 'ER',
 'lo',
 'sasha1',
 'ch',
 'rest',
 'john',
 'sta',
 'out',
 'st',
 'ne',
 'elcome',
 'we',
 'welcome',
 'wel',
 'nat',
 'mar',
 'ball',
 'tymrf',
 'ger',
 'ki',
 'king',
 'es',
 'ro',
 'happy',
 'ell',
 'flash',
 'qwe',
 'all',
 '12345',
 'th',
 'sa',
 'daniel',
 'ani',
 'shar',
 'jerr',
 'ken',
 'one',
 'fi',
 'fir',
 'firstone',
 '624',
 'ac',
 'mic',
 'art',
 'sha',
 'superm',
 'dd',
 'ista',
 'rist',
 '2000',
 'at',
 'scott',
 'in',
 'indi',
 'bz',
 'hfc',
 '987654321',
 '54321',
 'ton',
 'fgf',
 'gf',
 'fg',
 'kocha',
 'oc',
 'land',
 'home',
 's1',
 'good',
 'iper',
 'pers',
 'ipers',
 'per',
 'vi',
 'viper',
 'ett',
 'et',
 'ob',
 'bb',
 'obby',
 'y1',
 'bby1',
 'rob',
 'robb',
 'robby',
 'nokia',
 'batman',
 'aezakmi',
 'a123',
 'izard',
 'fuck',
 'min',
 'esca',
 'get',
 'gettysbu',
 'nder',
 'ion',
 'mi',
 'ud',
 'rude',
 'rud',
 'ru',
 'boy',
 'mari',
 'emon',
 'big',
 'rf',
 'ens',
 'green',
 'gre',
 'ford',
 'ord',
 'kam',
 'bla',
 'blast',
 'bl',
 'ster',
 'laster',
 'las',
 '111',
 'ham',
 'ture',
 'lar',
 'ola',
 'solar',
 'sol',
 'so',
 'tn',
 'ghbdtn',
 'ghb',
 'bd',
 'dtn',
 'nbr',
 'nb',
 'ey',
 'rand',
 'mme',
 'mmer',
 'tim',
 'ti',
 'im',
 'mer',
 'immer',
 'sun',
 'llin',
 'ossi',
 'lu',
 'zxcvb',
 'jxrf',
 'ama',
 'mada',
 'ada',
 'sex',
 'ver',
 'ter',
 'hel',
 'dragon',
 'a19',
 '95',
 '199',
 '1995',
 'vova199',
 'vova',
 'purple',
 'cat',
 'fucks',
 'tan',
 'black',
 'RE',
 'che',
 'access',
 'dolphin',
 'hin',
 'ret',
 'barr',
 'je',
 'mou',
 'oune',
 'll',
 're',
 'ant',
 'bu',
 'bal',
 'ian',
 'anthon',
 'on1',
 'nt',
 'nth',
 'anth',
 '1Cccccc',
 'Ccccc',
 'atthew',
 'matt',
 'ja',
 'am',
 'sam',
 'ock',
 'sea',
 'oliver',
 'oliv',
 'er2',
 'itsu',
 'piderman',
 'ide',
 'dee',
 'smith',
 'nch',
 'no00',
 'benno',
 'cks',
 'ero',
 'us',
 'ven',
 'a2010',
 '55555',
 'secret',
 'swe',
 'td',
 'eanin',
 'soccer',
 'un',
 'junior',
 'age',
 'verag',
 'tig',
 'mother',
 'smo',
 'the',
 'her',
 'moth',
 'fgh',
 'nin',
 'harley',
 'ent',
 'red',
 'ward',
 'er1',
 'sana',
 'os',
 'hea',
 'he',
 'ik',
 'kik',
 'kok',
 'con',
 'puss',
 'rocket',
 'rock',
 'cola',
 'col',
 '00',
 'dle',
 'mod',
 'ov',
 'casa',
 'asanov',
 'anov',
 'anova',
 'eni',
 'is',
 'is1',
 'ight',
 'averic',
 'eric',
 'ric',
 'maveric',
 'CO',
 'OC',
 'win',
 'ed',
 'gan',
 'allard',
 'eg',
 'ta',
 'ine',
 'bel',
 'lous',
 'horny',
 'hor',
 'yme',
 'me',
 'nascar',
 'andalf',
 'gandalf',
 'dal',
 'li',
 'april2',
 '147',
 'ile',
 'le',
 'fu',
 'ku',
 'ya',
 'yak',
 'ip',
 'ul',
 'andy',
 'ena',
 'girl',
 'lexandr',
 'music',
 'pas',
 'bob',
 'lovejo',
 'mike',
 'password',
 'passw',
 'ey2010',
 'ashi',
 'itch',
 'my',
 'schnu',
 'nuff',
 'la',
 'thug',
 'four',
 'ble',
 'ghjg',
 'ghjghj',
 'ghj',
 'stuff',
 'ove',
 'qaz',
 'mmy',
 'birthday',
 'pol', 'ook', 'eagle', '2wsx', 'par', 'wayne', 'apple', 'DICK', 'IC', 'hris', 'anger', 'tcast', 'tca', 'tcas', 'ban', 'ustin', 'nchester', 'field', 'olf', 'silver', 'xxx', 'dra', 'rat', 'bull', 'amir', 'mon', 'rus', 'ito','gri', 'negri', 'negrit', 'negr', 'hilli', 'chill', 'chillin', 'hello', 'night','ic']
# Function to categorize password
def password(v):
    
    # the password should not be a
    # newline or space
    if v == "\n" or v == " ":
        return "Password cannot be a newline or space!"
    # the password length should be in
    # between 9 and 20
    if 9 <= len(v) <= 20:
        #checks for patterns in password
        for pattern in patterns:
            if pattern in v:
                return "Weak Password: Weak patterns have been recognized in the password"
        # checks for occurrence of a character 
        # three or more times in a row
        if re.search(r'(.)\1\1', v):
            return "Weak Password: Same character repeats three or more times in a row"
        # checks for occurrence of same string 
        # pattern( minimum of two character length)
        # repeating
        if re.search(r'(..)(.*?)\1', v):
            return "Weak password: Same string pattern repetition"
        else:
            return "Strong Password!"
   
    else:
        return "Password length must be 9-20 characters!"
for p in passwords_generated:
    print(password(p))
print(password('love12394843'))

In [ ]:
patterns=['shadow', 'a199', 'dima199', '91', '1991', 'pun', 'EY', '1234', '123', 'eirf', 'nhei', 'dj', 'jhj', 'jy', 'way', 'amst', 'te', 'erda', 'sterda', 'al', 'ala', 'jordan','cho',
 'ol', 'evild', 'evi','evil','dead','ad','ea','vil','ld','000','blue','aaa','a2','str','moni','er','ojo11','mo','mojo','11','zxc','zxcv','love','br','bryan',
 'an','azz','ova','lan','rate','espera','esper','ca','on','su','ge','rna','rn','fur','ur','ace','butt','bur','ina','jack','auto','ill','tor','ector',
 'ecto','or','ect','ve','cto','ctor','and','fre','12','hot','ot','cial','pecial','pe','pi','19','oleg1','shir','sh','man','da','dais','ab','ba','angel',
 'arky','ra','ar','ake','ma','12345678','cooter','co','scooter','6chevy','killer','qwerty','ggy','master','ers','head','david','ing','ga','lon','ck',
 'dick','long','ng','count','ou','oun','ted','ire','oo','fire','artin','martin','in1','ker1','acker','23','poppy','pp','ppy123','be','zer','ee','thunder','dan',
 'ana','ho','ota','tac','vol','el','elvis','s99','elvi','ri','alex','se','hi','pimp','ER','lo','sasha1','ch','rest','john','sta','out','st','ne','elcome','we',
 'welcome','wel','nat','mar','ball','tymrf','ger','ki','king','es','ro','happy','ell','flash','qwe','all','12345','th','sa','daniel','ani','shar','jerr',
 'ken','one','fi','fir','firstone','624','ac','mic','art','sha','superm','dd','ista','rist','2000','at','scott','in','indi','bz','hfc','987654321','54321',
 'ton','fgf','gf','fg','kocha','oc','land','home','s1','good','iper','pers','ipers','per','vi','viper','ett','et','ob','bb','obby','y1','bby1','rob',
 'robb','robby','nokia','batman','aezakmi','a123','izard','fuck','min','esca','get','gettysbu','nder','ion','mi','ud','rude','rud','ru','boy','mari','emon',
 'big','rf','ens','green','gre','ford','ord','kam','bla','blast','bl','ster','laster','las','111','ham','ture','lar','ola','solar','sol','so','tn','ghbdtn',
 'ghb','bd','dtn','nbr','nb','ey','rand','mme','mmer','tim','ti','im','mer','immer','sun','llin','ossi','lu','zxcvb','jxrf','ama','mada','ada','sex','ver',
 'ter','hel','dragon','a19','95','199','1995','vova199', 'vova', 'purple', 'cat', 'fucks', 'tan', 'black', 'RE', 'che', 'access', 'dolphin', 'hin',
 'ret', 'barr', 'je', 'mou', 'oune', 'll', 're', 'ant', 'bu', 'bal', 'ian', 'anthon', 'on1', 'nt', 'nth', 'anth', '1Cccccc', 'Ccccc', 'atthew', 
 'matt', 'ja', 'am','sam', 'ock', 'sea', 'oliver', 'oliv', 'er2', 'itsu', 'piderman', 'ide', 'dee', 'smith', 'nch', 'no00', 'benno', 'cks', 'ero', 'us', 
 'ven', 'a2010', '55555', 'secret', 'swe', 'td', 'eanin', 'soccer', 'un', 'junior', 'age', 'verag', 'tig', 'mother', 'smo', 'the', 'her', 'moth', 
 'fgh', 'nin', 'harley', 'ent', 'red', 'ward', 'er1', 'sana', 'os', 'hea', 'he', 'ik', 'kik', 'kok', 'con', 'puss', 'rocket', 'rock', 'cola', 'col', 
 '00', 'dle', 'mod', 'ov', 'casa', 'asanov', 'anov', 'anova', 'eni', 'is', 'is1', 'ight', 'averic', 'eric', 'ric', 'maveric', 'CO', 'OC', 'win', 
 'ed', 'gan', 'allard', 'eg', 'ta', 'ine', 'bel', 'lous', 'horny', 'hor', 'yme', 'me', 'nascar', 'andalf', 'gandalf', 'dal', 'li', 'april2','147', 
 'ile', 'le', 'fu', 'ku', 'ya', 'yak', 'ip', 'ul', 'andy', 'ena', 'girl', 'lexandr', 'music', 'pas', 'bob', 'lovejo', 'mike', 'password', 'passw', 
 'ey2010', 'ashi', 'itch', 'my', 'schnu', 'nuff', 'la', 'thug', 'four', 'ble', 'ghjg', 'ghjghj', 'ghj', 'stuff', 'ove', 'qaz', 'mmy', 'birthday',
  'pol', 'ook', 'eagle', '2wsx', 'par', 'wayne', 'apple', 'DICK', 'IC', 'hris', 'anger', 'tcast', 'tca', 'tcas', 'ban', 'ustin', 'nchester',
   'field', 'olf','silver', 'xxx', 'dra', 'rat', 'bull', 'amir', 'mon', 'rus', 'ito', 'gri', 'negri','negrit','negr','hilli','chill','chillin',
   'hello','night','ic']

In [10]:
from nltk.metrics import distance
import scipy.spatial as spatial
import numpy as np
from scipy.cluster.vq import kmeans

# sample vocabulary list
#words = ['test', 'text', 'best', 'fast', 'context', 'boost', 'faster', 'border']
words=common_password['Common Passwords'].tolist()
words=words[:1000]
words=np.asarray(words)
# similarity matrix
word_vectors = np.array([
    [
        distance.edit_distance(w, _w)
        for _w in words
    ]
    for w in words
], dtype=np.float)

centroids, _ = kmeans(word_vectors, k_or_guess=100)

word_clusters = np.argmin([
    [spatial.distance.euclidean(wv, cv) for cv in centroids]
    for wv in word_vectors
], 1)

for k in range(centroids.shape[0]):
    print('k =', k)
    print([word for i, word in enumerate(words) if word_clusters[i] == k])

KeyboardInterrupt: 

In [12]:
def w1(s):
    c=0
    for char in s:
        if char.isalpha():
            c+=1
    return c*4
def w2(s):
    c=0
    for char in s:
        if char.islower():
            c+=1
    return (len(s)-c)*2
def w3(s):
    c=0
    for char in s:
        if char.isupper():
            c+=1
    return (len(s)-c)*2
def w4(s):
    c=0
    for char in s:
        if char.isdigit():
            c+=1
    return c*4
def w5(s):
    c=0
    for char in s:
        if not (char.isalpha() or char.isdigit()):
            c+=1
    return c*6
def w6(s):
    c=0
    for char in s[1:-1]:
        if char.isdigit():
            c+=1
        elif char.isalpha():
            pass
        else:
            c+=1
    return c*2
def w7(s):
    if s.isalpha():
        return -len(s)
    else:
        return 0
def w8(s):
    if s.isdigit():
        return -len(s)
    else:
        return 0
def w9(s):
    d,c={},0
    for char in s:
        if char not in d:
            d[char]=1
        else:
            d[char]+=1
    for char in d:
        if d[char]>1:
            c+=1
    return -(c*(c-1))

common_password["w1"]=common_password["Common Passwords"].apply(w1)
common_password["w2"]=common_password["Common Passwords"].apply(w2)
common_password["w3"]=common_password["Common Passwords"].apply(w3)
common_password["w4"]=common_password["Common Passwords"].apply(w4)
common_password["w5"]=common_password["Common Passwords"].apply(w5)
common_password["w6"]=common_password["Common Passwords"].apply(w6)
common_password["w7"]=common_password["Common Passwords"].apply(w7)
common_password["w8"]=common_password["Common Passwords"].apply(w8)
common_password["w9"]=common_password["Common Passwords"].apply(w9)
cols=["w1","w2","w3","w4","w5","w6","w7","w8","w9"]
common_password["score"]=common_password[cols].sum(axis=1)
common_password.to_csv('common_passwords.csv')

Making Trie Class

In [14]:
class TrieNode:
    def __init__(self):
        self.children={}
        self.endOfWord = False
class Trie:
    def __init__(self):
        self.root = TrieNode()
    def insert(self,word):
        cur = self.root
        for c in word:
            if c not in cur.children:
                cur.children[c] = TrieNode()
            cur=cur.children[c]
        cur.endOfWord = True
    def search(self,word):
        cur = self.root
        for c in word:
            if c not in cur.children:
                return False
            cur = cur.children[c]
        return cur.endOfWord

In [16]:
breached_passwds=Trie()
import os
l=os.getcwd()+"\\Leaked Database"
a=os.listdir(l)
for i in a:
    p=l+"\\"+i
    file = open(p, 'r',errors='ignore')
    for j in file:
        breached_passwds.insert(j)



KeyboardInterrupt: 

In [ ]:
'''import re
print("Hello world")
matches = re.findall(r'(..+)(?=.*\1)',"##bsdeifa###b")
largest = '' if not matches else max(matches,key=lambda m:len(m))
print(largest)
# Python 3 program to find the longest repeated
# non-overlapping substring
 
# Returns the longest repeating non-overlapping
# substring in str
def longestRepeatedSubstring(str):
 
    n = len(str)
    LCSRe = [[0 for x in range(n + 1)]
                for y in range(n + 1)]
 
    res = "" # To store result
    res_length = 0 # To store length of result
 
    # building table in bottom-up manner
    index = 0
    for i in range(1, n + 1):
        for j in range(i + 1, n + 1):
             
            # (j-i) > LCSRe[i-1][j-1] to remove
            # overlapping
            if (str[i - 1] == str[j - 1] and
                LCSRe[i - 1][j - 1] < (j - i)):
                LCSRe[i][j] = LCSRe[i - 1][j - 1] + 1
 
                # updating maximum length of the
                # substring and updating the finishing
                # index of the suffix
                if (LCSRe[i][j] > res_length):
                    res_length = LCSRe[i][j]
                    index = max(i, index)
                 
            else:
                LCSRe[i][j] = 0
 
    # If we have non-empty result, then insert
    # all characters from first character to
    # last character of string
    if (res_length > 0):
        for i in range(index - res_length + 1,
                                    index + 1):
            res = res + str[i - 1]
 
    return res
 
# Driver Code
if __name__ == "__main__":
     
    str = "geeksforgeeks"
    print(longestRepeatedSubstring(str))
 
# This code is contributed by ita_c'''

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from nltk.metrics import edit_distance

In [2]:
# Scraping Website 1
url1 = "https://en.wikipedia.org/wiki/Wikipedia:10,000_most_common_passwords"
webdata  = requests.get(url1).text
soup = BeautifulSoup(webdata,"html.parser")
cols = soup.find_all('div',class_='div-col')
data = {"Common Passwords":[]}
for col in cols:
    passwds=col.find_all('li')
    for passwd in passwds:
        if passwd.text.strip()!='':
            data["Common Passwords"].append(passwd.text)
#Scraping Website 2
url2 = 'https://github.com/OWASP/passfault/blob/master/wordlists/wordlists/10k-worst-passwords.txt'
webdata  = requests.get(url2).text
soup = BeautifulSoup(webdata,"html.parser")
passwds = soup.find_all('td',class_="blob-code blob-code-inner js-file-line")
for passwd in passwds:
    data["Common Passwords"].append(passwd.text)
#Scraping Website 3
url3='https://github.com/danielmiessler/SecLists/blob/master/Passwords/Common-Credentials/10-million-password-list-top-100000.txt'
webdata = requests.get(url3).text
soup = BeautifulSoup(webdata,"html.parser")
passwds = soup.find_all('td',class_='blob-code blob-code-inner js-file-line')
for passwd in passwds:
    data["Common Passwords"].append(passwd.text)    

#Writing all unique passwrods into a csv file    
common_password = pd.DataFrame.from_dict(data)
common_password = common_password[~common_password['Common Passwords'].str.isdigit()]
common_password = common_password.drop_duplicates(subset='Common Passwords', keep="first")
common_password = common_password.reset_index(drop=True)
common_password = common_password.sample(frac=1)
common_password.to_csv('common_passwords.csv')
words=common_password['Common Passwords'].tolist()
print(len(words))

62263


In [3]:
import numpy as np
from sklearn.cluster import AffinityPropagation
import distance
    
words = "apple bapple banana lapple anana".split(" ") #Replace this line
words = np.asarray(words) #So that indexing with a list will work
print(words)
lev_similarity = -1*np.array([[distance.levenshtein(w1,w2) for w1 in words] for w2 in words])

affprop = AffinityPropagation(affinity="precomputed", damping=0.999999999)
affprop.fit(lev_similarity)
exemplars=[]
for cluster_id in np.unique(affprop.labels_):
    exemplar = words[affprop.cluster_centers_indices_[cluster_id]]
    
    cluster = np.unique(words[np.nonzero(affprop.labels_==cluster_id)])
    cluster_str = ", ".join(cluster)
    print(" - *%s:* %s" % (exemplar, cluster_str))


['apple' 'bapple' 'banana' 'lapple' 'anana']
 - *apple:* apple, bapple, lapple
 - *anana:* anana, banana


In [4]:
exemplars=[words[affprop.cluster_centers_indices_[cluster_id]] for cluster_id in np.unique(affprop.labels_)]
print(exemplars)
exemplars.append('kanana')
print(exemplars)
l_similarity = -1*np.array([[distance.levenshtein(w1,w2) for w1 in exemplars] for w2 in exemplars])
from sklearn.metrics.pairwise import cosine_similarity
x=cosine_similarity(l_similarity)
print(x)
for i in range(len(x[-1])-1):
    if x[-1][i]>0.9 :
         print(exemplars[i])

['apple', 'anana']
['apple', 'anana', 'kanana']
[[1.         0.1893885  0.12251278]
 [0.1893885  1.         0.95130299]
 [0.12251278 0.95130299 1.        ]]
anana


In [5]:
print(lev_similarity)

[[ 0 -1 -5 -1 -4]
 [-1  0 -4 -1 -5]
 [-5 -4  0 -5 -1]
 [-1 -1 -5  0 -5]
 [-4 -5 -1 -5  0]]


In [6]:
from sklearn.metrics.pairwise import cosine_similarity
print(cosine_similarity(lev_similarity))

[[1.         0.95348837 0.24219865 0.97279635 0.27945998]
 [0.95348837 1.         0.27945998 0.97279635 0.24219865]
 [0.24219865 0.27945998 1.         0.23718595 0.97014925]
 [0.97279635 0.97279635 0.23718595 1.         0.23718595]
 [0.27945998 0.24219865 0.97014925 0.23718595 1.        ]]


In [7]:
#clustering using AffinityPropagation
import numpy as np
from sklearn.cluster import AffinityPropagation
import distance
words=common_password['Common Passwords'].tolist()
words=words[:4000]
words = np.asarray(words) #So that indexing with a list will work
lev_similarity = -1*np.array([[distance.levenshtein(w1,w2) for w1 in words] for w2 in words])

affprop = AffinityPropagation(affinity="precomputed", damping=0.5)
affprop.fit(lev_similarity)
clusters=[]
for cluster_id in np.unique(affprop.labels_):
    clus=[]
    exemplar = words[affprop.cluster_centers_indices_[cluster_id]]
    clus.append(exemplar)
    cluster = np.unique(words[np.nonzero(affprop.labels_==cluster_id)])
    for p in cluster:
        if p!=exemplar:
            clus.append(p)
    clusters.append(clus)
    cluster_str = ", ".join(cluster)
    print(" - *%s:* %s" % (exemplar, cluster_str))

 - *blackdog:* babydog, beaudog, black10, blackdog, blackshadow, jakedog, slickdog
 - *golf1:* gb2312, golf1, golf18, golf72, golfboy, golfpro, golfvr6, grizzly1, noles, paolo1, wolf, wolf100, wolfpac
 - *enter:* 2enter, Winter1, century, cn42qj, denver7, elmer1, ender, energ, enter, ertert, extrem, gentle, kenney, kouter, oyster, penner, renner, ronster, senator1, sentnece, tender, united99, until
 - *libero:* amberdog, cicero, hibernia, jibXHQ, libero, libertas, lidstrom, liverpool, livestrong, viper666
 - *comicsans:* comicbookdb, comicsans, coolbeans, mohicans
 - *gfhjkl:* Gfhjkm1, cgfhnfr, dfghjk, g3ujwg, gfhjkbot, gfhjkl, gfhnbz, ghjcnjq, vjqgfhjkm, yfhrjnbrb
 - *mora:* 1for, dork, ezra, fonda, maura, mega, mockba, monami, mopar1, mora, mota, norcal, olya, rrrr, sara, sosa, umbra, zamora
 - *fluke:* 3CuDjZ, Blue, flore, floyd, flugel, fluke, flux, iluvsex, klesko, neuken, vfeukb, zulu44
 - *dfytxrf:* adfasdf, bdfyeirf, cjymrf, dfkthbq, dfytxrf, dkflbckfd, fy.nrf, ghjdthrf, gjkbyj

In [8]:
exemplars=[words[affprop.cluster_centers_indices_[cluster_id]] for cluster_id in np.unique(affprop.labels_)]
print(exemplars)

['blackdog', 'golf1', 'enter', 'libero', 'comicsans', 'gfhjkl', 'mora', 'fluke', 'dfytxrf', 'edwardcullen', 'devil', 'samba1', 'keith', 'dog123', 'jeffery', 'PUSSIES', 'TINA', 'master2', 'orion2', 'cumshots', 'cleaners', 'stell', 'lollo', 'earth1', 'deecee', 'blown', 'boston', 'bertha1', 'bowler', 'grow', 'arie', 'jacks', 'fallout3', 'deamon', 'gauss', 'soccer3', '2606642yra', 'scarred', 'eastwood', 'peaches', 'lucky1', 'copyright', 'firewire', 'berlin1', 'pantys', 'websolutionssu', 'blue25', 'michaell', 'nathal', 'bigtim', 'mala', 'fuckyou7', 'jena', 'mani', 'dragon23', 'eremin', 'nation', '8PHroWZ622', 'littlecunt', 'poop69', 'josiah', 'melony', 'korea', 'iloveyou2', 'bmwm3', 'beto', 'unicron', 'alex1998', 'pookie', 'sigma', 'entertainment', 'sabado', 'hogman', 'Rusty1', 'tadlock', 'cfvfzcxfcnkbdfz', 'h_froeschl7', 'stangs', 'petros', 'cristan', 'kimota', 'ronnie1', 'MADMAN', 'grammar', 'fyutkbyrf', 'hjvfyjdf', 'bobette', 'fishon', 'BEER', 'picker', 'artboy', 'banan', 'CARTER', 'late

In [24]:
def longest_Substring(s1,s2):
    
    seq_match = SequenceMatcher(None,s1,s2) 
    match = seq_match.find_longest_match(0, len(s1), 0, len(s2)) 
    # return the longest substring 
    if (match.size!=0): 
        return (s1[match.a: match.a + match.size])  
    else: 
        return ''
exemplars=[words[affprop.cluster_centers_indices_[cluster_id]] for cluster_id in np.unique(affprop.labels_)]
#print(exemplars)
word='apple'
exemplars.append(word)
#print(exemplars)
l_similarity = -1*np.array([[distance.levenshtein(w1,w2) for w1 in exemplars] for w2 in exemplars])
from sklearn.metrics.pairwise import cosine_similarity
x=cosine_similarity(l_similarity)
print(x)
for i in range(len(x[-1])-1):
    if x[-1][i]>0.9 :
        if len(longest_Substring(word,exemplars[i]))>2:
            print(exemplars[i])

[[1.         0.97577677 0.96878414 ... 0.97279773 0.98187809 0.97554326]
 [0.97577677 1.         0.98172044 ... 0.98657802 0.97392287 0.98936217]
 [0.96878414 0.98172044 1.         ... 0.98052249 0.98349383 0.98409239]
 ...
 [0.97279773 0.98657802 0.98052249 ... 1.         0.96929403 0.99459648]
 [0.98187809 0.97392287 0.98349383 ... 0.96929403 1.         0.97338951]
 [0.97554326 0.98936217 0.98409239 ... 0.99459648 0.97338951 1.        ]]
couple
lapper
Purple


In [22]:
affprop = AffinityPropagation(damping=0.5)
affprop.fit(lev_similarity)
clusters=[]
for cluster_id in np.unique(affprop.labels_):
    clus=[]
    exemplar = words[affprop.cluster_centers_indices_[cluster_id]]
    clus.append(exemplar)
    cluster = np.unique(words[np.nonzero(affprop.labels_==cluster_id)])
    for p in cluster:
        if p!=exemplar:
            clus.append(p)
    clusters.append(clus)
    cluster_str = ", ".join(cluster)
    print(" - *%s:* %s" % (exemplar, cluster_str))

C:\Users\yoga\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\cluster\_affinity_propagation.py:236: ConvergenceWarning: Affinity propagation did not converge, this model may return degenerate cluster centers and labels.
  warnings.warn(


IndexError: list index out of range

In [10]:
clusters #Nested list in which each element contains cluster of passwords
temp=clusters #creating a copy of clusters to work on

In [11]:
#Finding different patterns present in each cluster and the frequency of these patterns
from difflib import SequenceMatcher 
def longest_Substring(s1,s2):
    
    seq_match = SequenceMatcher(None,s1,s2) 
    match = seq_match.find_longest_match(0, len(s1), 0, len(s2)) 
    # return the longest substring 
    if (match.size!=0): 
        return (s1[match.a: match.a + match.size])  
    else: 
        return False
patternInClusters=[]
for i in range(len(temp)):
    patternInClusters.append({})
    exemplar=temp[i][0]
    for j in temp[i][1:]:
        x=longest_Substring(exemplar,j)
        if x!=False and len(x)>2:
            if x in patternInClusters[i]:
                patternInClusters[i][x]+=1
            else:
                patternInClusters[i][x]=1

patternInClusters

[{'dog': 3, 'black': 2, 'ckdog': 1},
 {'golf1': 1, 'golf': 4, 'olf': 2, 'olf1': 1},
 {'enter': 1, 'nter': 1, 'ent': 3, 'ter': 4},
 {'ber': 1, 'ero': 1, 'iber': 1, 'liber': 1},
 {'comic': 1, 'ans': 2},
 {'fhjk': 1, 'gfh': 2, 'hjk': 1, 'gfhjk': 2},
 {'mora': 1},
 {'flu': 2, 'uke': 1},
 {'dfy': 1, 'xrf': 1, 'txrf': 1},
 {},
 {'evil': 2, 'devil': 2, 'devi': 1, 'evi': 2},
 {'amb': 2, 'sam': 2, 'mba': 2},
 {'ith': 3, 'keith': 1},
 {'123': 7, 'g123': 1, 'dog1': 1},
 {'jeffer': 1, 'jeff': 2, 'eff': 1},
 {'PUSS': 1},
 {'TIN': 1},
 {'ster': 4, 'aste': 1, 'ast': 1, 'master': 5, 'ster2': 1, 'ter': 1},
 {'ion2': 1, 'orion': 2},
 {'shots': 1},
 {'clea': 1, 'lean': 1, 'ers': 1},
 {'stell': 1, 'tell': 1, 'ell': 5, 'stel': 1, 'ste': 3},
 {'ollo': 3, 'lol': 3, 'oll': 3},
 {'th1': 2, 'art': 1, 'ear': 1, 'earth': 1},
 {'ece': 1, 'dee': 2},
 {'blo': 2, 'blow': 1, 'own': 2},
 {'oston': 1, 'bos': 2, 'boston': 1, 'ton': 2, 'ost': 1, 'osto': 1, 'ston': 1},
 {'ber': 1, 'bertha': 1, 'tha': 1, 'ha1': 1, 'ert': 1}

In [12]:
#adding the most common patterns in each cluster to a final pattern list
patterns=[]
for sampleDict in patternInClusters:
    if sampleDict=={}:
        continue
    itemMaxValue = max(sampleDict.items(), key=lambda x: x[1])
    #print('Maximum Value in Dictionary : ', itemMaxValue[1])
    listOfKeys = list()
    # Iterate over all the items in dictionary to find keys with max value
    for key, value in sampleDict.items():
        if value == itemMaxValue[1]:
            listOfKeys.append(key)
    for i in listOfKeys:
        if i not in patterns:
            patterns.append(i)
patterns

['dog',
 'golf',
 'ter',
 'ber',
 'ero',
 'iber',
 'liber',
 'ans',
 'gfh',
 'gfhjk',
 'mora',
 'flu',
 'dfy',
 'xrf',
 'txrf',
 'evil',
 'devil',
 'evi',
 'amb',
 'sam',
 'mba',
 'ith',
 '123',
 'jeff',
 'PUSS',
 'TIN',
 'master',
 'orion',
 'shots',
 'clea',
 'lean',
 'ers',
 'ell',
 'ollo',
 'lol',
 'oll',
 'th1',
 'dee',
 'blo',
 'own',
 'bos',
 'ton',
 'bertha',
 'tha',
 'ha1',
 'ert',
 'ler',
 'gro',
 'rie',
 'jack',
 'allo',
 'fallout',
 'out',
 'amon',
 'uss',
 'soccer',
 'carr',
 'scar',
 'sca',
 'wood',
 'ast',
 'eaches',
 'uck',
 'ight',
 'fire',
 'ewire',
 'ant',
 'pant',
 'panty',
 'blue',
 'michael',
 'atha',
 'natha',
 'ath',
 'nathal',
 'big',
 'ala',
 'fuck',
 'fuckyou',
 'jen',
 'man',
 'dragon',
 'rem',
 'ation',
 'little',
 'poo',
 'poop',
 'jos',
 'mel',
 'kor',
 'ilove',
 'bmw',
 'bmwm',
 'icron',
 'unic',
 'uni',
 'alex199',
 '199',
 'ookie',
 'sigma',
 'ado',
 'bado',
 'aba',
 'Rus',
 'ty1',
 'ust',
 'usty',
 'ock',
 'lock',
 'sta',
 'ros',
 'ist',
 'kim',
 'nie

In [2]:
#Implementing Algorithm to generate passwords
import random
words = input("Enter five words(give space between each word): ").split()
numbers = input("Enter two numbers(give space between each number): ").split()
num_passwords = int(input("Enter number of passwords required:"))
special_chars = ['@','$','!','#','%','&','(',')','0','3','8','<','|']
punc = ['@','*','+','-',':','”','/','\\','~','?','[',']','{','}','$','!','#','%','&','(',')','_' ,'<','|']
alphabets = ['a','s','i','r','x','q','c','j','o','e','b','k','l']
c2s = {}
for i in range(13):
    c2s[alphabets[i]]=special_chars[i]
def randomly_capitalize(s):
    result = ''
    for char in s:
        flag = random.randint(0, 1)
        if flag == 0:
            result += char.upper()
        else:
            result += char.lower()
    return result
def merge_randomly(str1,str2):
    result = ''
    flag = random.randint(0, 1)
    if flag == 0:
        result = str1 + str2
    else:
        result = str2 + str1
    return result,flag
def charToSpecial(s):
    for char in s:
        if char in c2s:
            s = s.replace(char,c2s[char])
    return s
def insert_number(s,n,f,str1,str2):
    flag = random.randint(0, 2)
    if flag == 0:
        return n+s
    elif flag == 2:
        return s+n
    else:
        if f==0:
            return s[:len(str1)]+n+s[len(str1):]
        else:
            return s[:len(str2)]+n+s[len(str2):]
def add_punctuation(s):
    return s+random.choice(punc)
passwords_generated=[]
while num_passwords>0:
    temp = list(words)
    string1 = random.choice(temp)
    temp.remove(string1)
    string2 = random.choice(temp)
    number = random.choice(numbers)
    string1,string2 = randomly_capitalize(string1),randomly_capitalize(string2)
    final_string,f = merge_randomly(string1,string2)
    final_string = charToSpecial(final_string)
    final_string = insert_number(final_string,number,f,string1,string2)
    final_string = add_punctuation(final_string)
    C = len(final_string)
    if C < 8:
        for i in range(8 - C):
            final_string += random.choice(special_chars)
    passwords_generated.append(final_string)
    print("Final Passowrd:",final_string)
    num_passwords -= 1

KeyboardInterrupt: Interrupted by user

In [4]:
#To analyze given/generated password
import re

# Function to categorize password
def password(v):
    # the password should not be a
    # newline or space
    if v == "\n" or v == " ":
        return "Password cannot be a newline or space!"
    # the password length should be in
    # between 9 and 20
    if 9 <= len(v) <= 20:
        #checks for patterns in password
        for pattern in patterns:
            if pattern in v:
                return "Weak Password: Weak patterns have been recognized in the password"
        # checks for occurrence of a character 
        # three or more times in a row
        if re.search(r'(.)\1\1', v):
            return "Weak Password: Same character repeats three or more times in a row"
        # checks for occurrence of same string 
        # pattern( minimum of two character length)
        # repeating
        if re.search(r'(..)(.*?)\1', v):
            return "Weak password: Same string pattern repetition"
        else:
            return "Strong Password!"
    else:
        return "Password length must be 9-20 characters!"
#Analyzing strength of generated passwords: 
#for p in passwords_generated:
#    print(password(p))
#Analyzing strength of password given by user: 
print(password('123456789'))

Weak Password: Weak patterns have been recognized in the password


In [30]:
words=common_password['Common Passwords'].tolist()
words=words[:4000]
words = np.asarray(words) #So that indexing with a list will work
le= -1*np.array([distance.levenshtein('hello',w1) for w1 in words])
affprop.predict(le)
print(lev_similarity)
print(len(lev_similarity))

ValueError: Expected 2D array, got 1D array instead:
array=[ -6  -5  -7 ... -15  -8  -3].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [5]:
a=['gan',
 'ure',
 'kata',
 'rina',
 'kat',
 'rin',
 'west',
 'she',
 'incognit',
 'illie',
 'bill',
 'lineage',
 'age123',
 'ali',
 'emon',
 'crack',
 'gen',
 'ilove',
 'ie1',
 'times',
 'mes',
 'what',
 'miss',
 'wwwww',
 'brandon',
 'brave',
 'med',
 'mohamme',
 'hamm',
 'daniel',
 'ley',
 'y12',
 'ard',
 'dor',
 'the',
 'leedsu',
 'iddqd8',
 'wood',
 'micr',
 'bug',
 'ladybug',
 'buffy',
 'uffy',
 'sophie',
 'laketaho',
 'ket',
 'pic',
 'man',
 'onal',
 'rona',
 'nald',
 'way',
 'ent',
 'sarah',
 'ton',
 'jgj',
 'gjhn',
 'ste',
 'ucks',
 'let',
 'key',
 'fuck',
 'one',
 'ner',
 'oner',
 'loo',
 'georg',
 'erchief',
 'master',
 'ick',
 'alex',
 'ania',
 'nia',
 'ani',
 '123456789',
 '123456',
 'ELE',
 '111111',
 'erico',
 'sych',
 'psych',
 'anthon',
 'ant',
 'ny1',
 'ierr',
 'red',
 'fre',
 'fred',
 'heel',
 'tar',
 'tarheel',
 'hee',
 'fktrc',
 'fkt',
 'petite',
 'etit',
 'eti',
 'bart',
 'art',
 'ette',
 'laudi',
 'claudiu',
 'lau',
 'mar',
 'tor',
 'victo',
 'tori',
 'mpton',
 'hampton',
 'pok',
 'omas',
 'toma',
 'tone',
 'get',
 'geto',
 'etone',
 'dfcbk',
 'dfcbkmtd',
 'asdfgh12',
 'asdfgh',
 'asd',
 'angel',
 'golfer',
 'golfer2',
 'golf',
 'fer',
 'nny',
 'simon',
 'ock',
 'aze',
 'zer',
 'oof',
 'chris',
 'ris',
 'ewa',
 'pro',
 'ron',
 'ipper',
 'ets',
 'hyper',
 'ter123',
 'azz',
 'ster',
 'big',
 'lat',
 'boob',
 'ers',
 'start',
 'art1',
 'sta',
 'star',
 'ippo',
 'hippo',
 'ipp',
 'uke',
 'jose',
 'teddy',
 'teddyb',
 'ONNIE',
 'lan',
 'ANN',
 'ANNON',
 'dre',
 'reyf',
 'password',
 'min',
 'egirl',
 'girl',
 'ceg',
 'surfin',
 'lvbnhb',
 'a20',
 '2002',
 'newpass',
 'new',
 'gged',
 'agge',
 'ged',
 'pets',
 'trumpet',
 'sco',
 'nger',
 'bur',
 'blue',
 'andy',
 'run',
 'illy',
 'lly',
 'kers',
 'sun',
 'ball',
 'rri',
 'mor',
 'rocket',
 'goc',
 'edward',
 'harlie',
 'charlie',
 'charl',
 'iss',
 'bastar',
 'meri',
 'omm',
 'mmy',
 'momm',
 '123',
 'cab',
 'ible',
 'indi',
 'head',
 'edskins',
 '2010',
 'erlin',
 'erlin1',
 'lin1',
 'merlin',
 'in1',
 'lin',
 'er1',
 'good',
 'asa',
 'asan',
 'mer',
 'ephan',
 'tephanie',
 'cruise',
 'xplo',
 'explo',
 'ally',
 'apple',
 'apples',
 'appl',
 'les',
 'app',
 'str',
 'shan',
 'shane1',
 'shane',
 'sha',
 'ari',
 'ustin',
 'fish1',
 'fish',
 'ish1',
 'char',
 'cha',
 'ain',
 'bos',
 'un1',
 'mike',
 'cobra',
 'weapon',
 'noo',
 'snoo',
 'aaaa',
 'aaa',
 'aaaaa',
 'qwert',
 'qwerty',
 'qwer',
 'dog',
 'rea',
 'kore',
 'kor',
 '1234',
 'music',
 'katie',
 'ati',
 'layer',
 'pla',
 '1qaz',
 'qqqq',
 'qqq',
 '5150',
 'der',
 'ander',
 'ing',
 'ora',
 'prior',
 'pri',
 'SCO',
 'sexy',
 'atali',
 'tali',
 'sm4llvil',
 'ena',
 'foot',
 'qwertyuiop12345',
 'cele',
 'lit',
 'kis',
 'rry',
 'erry',
 '54321',
 'aba',
 'dad',
 'cnbr',
 'nbr',
 'birthday',
 'yjdf',
 'jdf',
 'roman',
 'john',
 'imm',
 'can',
 'anon1',
 'non',
 'ell',
 'well',
 'iddle',
 'dles',
 'hoo',
 'ester',
 'fytxrf',
 'asdf',
 'asdfg',
 'cape',
 'ole',
 'pol',
 'abell',
 'hot',
 'tre',
 'hotr',
 'hol',
 'qazwsxedc1',
 'qazwsxedc',
 'bubb',
 'ite',
 'favorite',
 'avo',
 'fire',
 'kman',
 'uck',
 'ckman',
 'che',
 'ar1',
 'celtic',
 'elt',
 'black',
 'ara',
 'ova',
 'camel',
 'came',
 'cam',
 'ela',
 'mnbvcxz',
 'vert',
 'pervert',
 'joe',
 'losangel',
 'ero',
 'cool',
 'ool',
 'boy',
 'ERS',
 'diamond',
 'ORD',
 'squi',
 'olo',
 '12345',
 'werty1234',
 'werty12345',
 'oron',
 'ronto',
 'toro',
 'tto',
 'james',
 'er22',
 'tiger',
 'madelein',
 'del',
 'RIA',
 'MARI',
 'MAR',
 'ugs',
 'dragon',
 'night',
 'cali',
 'kin',
 'onnie',
 'nnie',
 'ord',
 'eggie',
 'gie',
 'reg',
 'ller',
 'alle',
 '200',
 'ter',
 'oun',
 'ever',
 'jesus',
 'sweet',
 'krist',
 '777',
 'corpio',
 'orpio',
 'ach',
 'en1',
 'even1',
 'anderso',
 'erso',
 'ders',
 'anders',
 'blood',
 'lood',
 'banan',
 'david',
 'tha',
 'abet',
 'elizabe',
 'bet',
 'ford',
 'inte',
 'zxc',
 'wil',
 'uddy',
 'son',
 'mag',
 'guy',
 'pan',
 'abela',
 'bab',
 'land',
 'atch',
 'long',
 'zzzz1',
 'zzzz',
 'ill',
 'maxim',
 'maximil',
 'imi',
 'oulou',
 'ulou',
 'mou',
 'oul',
 'ran',
 'soccer',
 'ashley',
 'bingo',
 'ack',
 'bon',
 'fffff',
 'peter',
 '1995',
 'dima',
 'dima19',
 '199',
 'a19',
 'a1995',
 'alesh',
 'ales',
 'lesh',
 'esha',
 'ker',
 'super',
 'tro',
 'oot',
 'ootball',
 'BIG',
 'atl',
 'andro',
 'sand',
 'sandr',
 'san',
 'bigd',
 'bigdu',
 'olphin1',
 'phin',
 'orno',
 'fhtym',
 'gfh',
 'ling',
 'ili',
 'free',
 'sus',
 'sex',
 'a198',
 'anna19',
 '1986',
 'minota',
 'des',
 'destro',
 'ico',
 'mustan',
 'guita',
 'ndrea',
 'bitch',
 'cent',
 'clement',
 'ment',
 'lon',
 'onl',
 'con',
 'abby',
 '234',
 '1q2',
 '2345',
 'ore',
 'horny',
 'champ',
 'gemini1',
 'emin',
 'ini11',
 'emi',
 'rill',
 'dio',
 'adi',
 'adio',
 'hole',
 'pa123',
 'a123',
 'ann',
 'nne',
 'ean',
 'ave',
 'jxrf',
 'har',
 'pee',
 'per',
 'ance',
 '3qq',
 '123123q',
 'linda',
 'martin',
 'shi',
 'umpy',
 '6666',
 'itty',
 'custom',
 'cus',
 'ust',
 'jack',
 'ass',
 'pass',
 'green',
 'call',
 'lla',
 'rami',
 'ett',
 'llllll',
 'line',
 'ome',
 'hello',
 'flb',
 'byf',
 'dar',
 'rrrr',
 'rrrrrr']
print(a)

['gan', 'ure', 'kata', 'rina', 'kat', 'rin', 'west', 'she', 'incognit', 'illie', 'bill', 'lineage', 'age123', 'ali', 'emon', 'crack', 'gen', 'ilove', 'ie1', 'times', 'mes', 'what', 'miss', 'wwwww', 'brandon', 'brave', 'med', 'mohamme', 'hamm', 'daniel', 'ley', 'y12', 'ard', 'dor', 'the', 'leedsu', 'iddqd8', 'wood', 'micr', 'bug', 'ladybug', 'buffy', 'uffy', 'sophie', 'laketaho', 'ket', 'pic', 'man', 'onal', 'rona', 'nald', 'way', 'ent', 'sarah', 'ton', 'jgj', 'gjhn', 'ste', 'ucks', 'let', 'key', 'fuck', 'one', 'ner', 'oner', 'loo', 'georg', 'erchief', 'master', 'ick', 'alex', 'ania', 'nia', 'ani', '123456789', '123456', 'ELE', '111111', 'erico', 'sych', 'psych', 'anthon', 'ant', 'ny1', 'ierr', 'red', 'fre', 'fred', 'heel', 'tar', 'tarheel', 'hee', 'fktrc', 'fkt', 'petite', 'etit', 'eti', 'bart', 'art', 'ette', 'laudi', 'claudiu', 'lau', 'mar', 'tor', 'victo', 'tori', 'mpton', 'hampton', 'pok', 'omas', 'toma', 'tone', 'get', 'geto', 'etone', 'dfcbk', 'dfcbkmtd', 'asdfgh12', 'asdfgh', 'a

In [ ]:
patterns=['shadow', 'a199', 'dima199', '91', '1991', 'pun', 'EY', '1234', '123', 'eirf', 'nhei', 'dj', 'jhj', 'jy', 'way', 'amst', 'te', 'erda', 'sterda', 'al', 'ala', 'jordan','cho',
 'ol', 'evild', 'evi','evil','dead','ad','ea','vil','ld','000','blue','aaa','a2','str','moni','er','ojo11','mo','mojo','11','zxc','zxcv','love','br','bryan',
 'an','azz','ova','lan','rate','espera','esper','ca','on','su','ge','rna','rn','fur','ur','ace','butt','bur','ina','jack','auto','ill','tor','ector',
 'ecto','or','ect','ve','cto','ctor','and','fre','12','hot','ot','cial','pecial','pe','pi','19','oleg1','shir','sh','man','da','dais','ab','ba','angel',
 'arky','ra','ar','ake','ma','12345678','cooter','co','scooter','6chevy','killer','qwerty','ggy','master','ers','head','david','ing','ga','lon','ck',
 'dick','long','ng','count','ou','oun','ted','ire','oo','fire','artin','martin','in1','ker1','acker','23','poppy','pp','ppy123','be','zer','ee','thunder','dan',
 'ana','ho','ota','tac','vol','el','elvis','s99','elvi','ri','alex','se','hi','pimp','ER','lo','sasha1','ch','rest','john','sta','out','st','ne','elcome','we',
 'welcome','wel','nat','mar','ball','tymrf','ger','ki','king','es','ro','happy','ell','flash','qwe','all','12345','th','sa','daniel','ani','shar','jerr',
 'ken','one','fi','fir','firstone','624','ac','mic','art','sha','superm','dd','ista','rist','2000','at','scott','in','indi','bz','hfc','987654321','54321',
 'ton','fgf','gf','fg','kocha','oc','land','home','s1','good','iper','pers','ipers','per','vi','viper','ett','et','ob','bb','obby','y1','bby1','rob',
 'robb','robby','nokia','batman','aezakmi','a123','izard','fuck','min','esca','get','gettysbu','nder','ion','mi','ud','rude','rud','ru','boy','mari','emon',
 'big','rf','ens','green','gre','ford','ord','kam','bla','blast','bl','ster','laster','las','111','ham','ture','lar','ola','solar','sol','so','tn','ghbdtn',
 'ghb','bd','dtn','nbr','nb','ey','rand','mme','mmer','tim','ti','im','mer','immer','sun','llin','ossi','lu','zxcvb','jxrf','ama','mada','ada','sex','ver',
 'ter','hel','dragon','a19','95','199','1995','vova199', 'vova', 'purple', 'cat', 'fucks', 'tan', 'black', 'RE', 'che', 'access', 'dolphin', 'hin',
 'ret', 'barr', 'je', 'mou', 'oune', 'll', 're', 'ant', 'bu', 'bal', 'ian', 'anthon', 'on1', 'nt', 'nth', 'anth', '1Cccccc', 'Ccccc', 'atthew', 
 'matt', 'ja', 'am','sam', 'ock', 'sea', 'oliver', 'oliv', 'er2', 'itsu', 'piderman', 'ide', 'dee', 'smith', 'nch', 'no00', 'benno', 'cks', 'ero', 'us', 
 'ven', 'a2010', '55555', 'secret', 'swe', 'td', 'eanin', 'soccer', 'un', 'junior', 'age', 'verag', 'tig', 'mother', 'smo', 'the', 'her', 'moth', 
 'fgh', 'nin', 'harley', 'ent', 'red', 'ward', 'er1', 'sana', 'os', 'hea', 'he', 'ik', 'kik', 'kok', 'con', 'puss', 'rocket', 'rock', 'cola', 'col', 
 '00', 'dle', 'mod', 'ov', 'casa', 'asanov', 'anov', 'anova', 'eni', 'is', 'is1', 'ight', 'averic', 'eric', 'ric', 'maveric', 'CO', 'OC', 'win', 
 'ed', 'gan', 'allard', 'eg', 'ta', 'ine', 'bel', 'lous', 'horny', 'hor', 'yme', 'me', 'nascar', 'andalf', 'gandalf', 'dal', 'li', 'april2','147', 
 'ile', 'le', 'fu', 'ku', 'ya', 'yak', 'ip', 'ul', 'andy', 'ena', 'girl', 'lexandr', 'music', 'pas', 'bob', 'lovejo', 'mike', 'password', 'passw', 
 'ey2010', 'ashi', 'itch', 'my', 'schnu', 'nuff', 'la', 'thug', 'four', 'ble', 'ghjg', 'ghjghj', 'ghj', 'stuff', 'ove', 'qaz', 'mmy', 'birthday',
  'pol', 'ook', 'eagle', '2wsx', 'par', 'wayne', 'apple', 'DICK', 'IC', 'hris', 'anger', 'tcast', 'tca', 'tcas', 'ban', 'ustin', 'nchester',
   'field', 'olf','silver', 'xxx', 'dra', 'rat', 'bull', 'amir', 'mon', 'rus', 'ito', 'gri', 'negri','negrit','negr','hilli','chill','chillin',
   'hello','night','ic']

In [31]:
exemplars=['blackdog', 'golf1', 'enter', 'libero', 'comicsans', 'gfhjkl', 'mora', 'fluke', 'dfytxrf', 'edwardcullen', 'devil', 'samba1', 'keith', 'dog123', 'jeffery', 'PUSSIES', 'TINA', 'master2', 'orion2', 'cumshots', 'cleaners', 'stell', 'lollo', 'earth1', 'deecee', 'blown', 'boston', 'bertha1', 'bowler', 'grow', 'arie', 'jacks', 'fallout3', 'deamon', 'gauss', 'soccer3', '2606642yra', 'scarred', 'eastwood', 'peaches', 'lucky1', 'copyright', 'firewire', 'berlin1', 'pantys', 'websolutionssu', 'blue25', 'michaell', 'nathal', 'bigtim', 'mala', 'fuckyou7', 'jena', 'mani', 'dragon23', 'eremin', 'nation', '8PHroWZ622', 'littlecunt', 'poop69', 'josiah', 'melony', 'korea', 'iloveyou2', 'bmwm3', 'beto', 'unicron', 'alex1998', 'pookie', 'sigma', 'entertainment', 'sabado', 'hogman', 'Rusty1', 'tadlock', 'cfvfzcxfcnkbdfz', 'h_froeschl7', 'stangs', 'petros', 'cristan', 'kimota', 'ronnie1', 'MADMAN', 'grammar', 'fyutkbyrf', 'hjvfyjdf', 'bobette', 'fishon', 'BEER', 'picker', 'artboy', 'banan', 'CARTER', 'late', 'chane', 'spurs1', 'bigdadd', 'contest', 'foutre', '1q2w3', 'valenti', 'cooking1', 'hubble', 'silver69', 'money2', 'horatio', 'amelia', 'bobbbb', 'brand', 'dindin', 'pussy50', 'bruce', 'alpha1', 'azxcvbnm', 'rachel1', 'nhfdvfnjkju123', 'boogies', 'mooser', 'stoker', 'fred1', 'twisty', 'ireland', 'internal', 'david15', '3f3fphT7oP', 'merger', 'nine', 'nightime', 'severian', 'teenie', 'huangjin1987', 'revers', 'fallon', 'gopats', 'Kimberly', 'sanfran', 'titotito', 'zsxmr7sztmr', 'hello123', 'sally', 'boob', 'asd321', 'redhat', 'liao', 'dima1996', 'olaola', 'ballet', 'michigan', 'redsea', 'heller', 'hanson', 'lurcher', 'dude1', 'renard', 'dance', 'raser', 'elizabe', 'crazy4u', 'sexygirl', 'LAUREN', 'passwordpassword', 'den1020834880', 'aviles', 'munter', 'kenshin1', 'deaddead', '123456789a', 'wildcat7', 'mine11', 'newera', 'miras', 'kissmy', 'add123', 'ratcat', 'bureau', 'a12345', 'pelus', 'shocker1', 'forfar', 'audia6', '121212q', 'romana', 'ariana', 'tree1', 'seren', 'savant', 'hummer', 'moocow', 'feline', 'killer21', 'mcgill', 'luvluv', 'shop', 'solene', 'cunning', 'bunty123', 'vladimi', 'hither', 'ragger', 'killas', '1967gto', 'a7777777', 'Year2005', 'bubbas', 'austi', 'edcrfv', 'steph1', 'kassie', 'infest', 'alone', '111111d', 'crjhjcnm', 'sadie123', 'passion', 'marcelo1', 'sasha', 'hacke', 'wesley', 'thrush', 'sammyg', 'chester', 'malina', 'papas', 'empress', 'andrew', 'xxxp455w0rd5', 'chelly', 'couple', 'murano', 'gfhfif', 'steelroa', 'uhfvjnf', 'Guitar', 'bahama', 'lapper', 'qweasdzx', 'joker123', 'blueboy', 'tatonka', 'backbone', 'sachin', 'Phillip', 'poolboy', 'italia', 'elena1977', 'player22', 'alex1', 'ytdpkjvfti', 'explosiv', 'charoot', 'LOST4815162342', 'spide', 'werder', 'carlin', 'check1', 'qaz123', 'rosalie', 'maggie', 'altair', 'basses', 'george1', 'chanchal', 'mention', 'cottage1', 'socks', 'anton1992', 'caveman1', 'pokemon', 'taro', 'natasa', 'pier', 'airman', 'Hd764nW5d7E1vbv', 'pinpin', 'SCOOBY', 'honor', 'scorpio', 'hott', 'pirelli', 'ipo54tj45uy856', 'Yellow', 'ababagalamaga', 'ziff', '1234567z', 'tiggy1', 'constitution', 'gangsta', 'gthtrfnbgjkt', 'mama12', 'popova', 'fynfyfyfhbde', 'assa123', 'favorite', 'basketball1', 'cristo', 'payman', 'strats', 'runn', 'chiens', 'bigdon', 'beating', 'dudelove', 'lhbjkjubz2957704', 'picass', 'billing', 'LITTLE', 'jimbo1', 'canon1', 'pepper12', 'NICK1234-rem936', 'loser1', 'angeli', 'Californ', 'tootall', 'chipper1', 'darren', 'siege', 'junior8', 'alex2010', 'cheese2', 'franki', 'netscape', 'getin1', 'elliott', 'baltimore', 'springs', 'kakawka', '237081a', 'fink', 'chippy', 'butte', 'daniel20', 'mariya', 'janet', 'greentea', 'barman', 'elric1', 'motorhead', 'theta', 'thunder', 'mess', 'nikolay', 'freetime', 'honda2000', 'lickin', 'sexyboy', 'dustoff', 'fcnfkfdbcnf', 'vfiekmrf', 'john1', 'schwab', 'typer', '070793monolit', 'vitebsk', 'sampson1', 'Fffff1', 'INSTALLDEVIC', 'spanky', 'yfnecz', 'skate1', 'loveit', 'anutka', 'hardcore', 'shirt', 'Purple', 'imcool', 'banderos', 'hokies', 'bulldog', 'light2', '1234ab', 'papito', 'june23', 'beaks', 'banker', 'morgan', 'august20', '123qazwsxedc', 'burnside', 'huntsman', 'coors', 'beowolf', 'rifle', 'oneway', 'robotic', 'mather', 'emile', 'cascade', 'R030989', 'chrisi', 'cando', 'baba', 'maddy1', 'teeter', 'snack', 'linus1', 'rimini', 'punch', 'ready', 'deltas', 'conny', 'super', 'harry2', 'tiger00', 'conger', 'chimay', 'JASON', 'actors', 'polaris', 'famili', 'farley', 'merit', 'calle', 'westerns']
def word2vec(word):
    from collections import Counter
    from math import sqrt

    # count the characters in word
    cw = Counter(word)
    # precomputes a set of the different characters
    sw = set(cw)
    # precomputes the "length" of the word vector
    lw = sqrt(sum(c*c for c in cw.values()))

    # return a tuple
    return cw, sw, lw

def cosdis(v1, v2):
    # which characters are common to the two words?
    common = v1[1].intersection(v2[1])
    # by definition of cosine distance we have
    return sum(v1[0][ch]*v2[0][ch] for ch in common)/v1[2]/v2[2]
x='blackdog'
for i in range(len(exemplars)):
    if cosdis(word2vec('blackdog'),word2vec(exemplars[i]))>0.7:
        print(exemplars[i])

blackdog
tadlock
bulldog


In [2]:
print("Hello")

Hello


In [20]:
import pandas as pd
  
# read contents of csv file
file = pd.read_csv("breached.csv")
print("\nOriginal file:")
print(file)
  
# adding header
headerList = ['passwords']
  
# converting data frame to csv
file.to_csv("breached.csv", header=headerList, index=False)
  
# display modified csv file
file2 = pd.read_csv("breached.csv")
print('\nModified file:')
print(file2)


Original file:
           123456
0        password
1        12345678
2            1234
3           pussy
4           12345
...           ...
1349834   yankees
1349835    yellow
1349836    zxcvbn
1349837   zxcvbnm
1349838    zzzzzz

[1349839 rows x 1 columns]

Modified file:
        passwords
0        password
1        12345678
2            1234
3           pussy
4           12345
...           ...
1349834   yankees
1349835    yellow
1349836    zxcvbn
1349837   zxcvbnm
1349838    zzzzzz

[1349839 rows x 1 columns]


In [23]:
import pandas as pd
df=pd.read_csv("breached.csv")
df.describe


<bound method NDFrame.describe of         passwords
0        password
1        12345678
2            1234
3           pussy
4           12345
...           ...
1349834   yankees
1349835    yellow
1349836    zxcvbn
1349837   zxcvbnm
1349838    zzzzzz

[1349839 rows x 1 columns]>

In [28]:
display(df.drop_duplicates())

,passwords
0,password
1,12345678
2,1234
3,pussy
4,12345
...,...
1349464,!1251llm
1349465,!!sonja!!
1349466,!!goto69
1349467,!!65logan


In [42]:
pwds=df['passwords'].tolist()
pwds
print('araceli=C3=A7' in pwds)

True


In [26]:
print(df.dtypes)

passwords    object
dtype: object


In [25]:
df=df.astype('str')